# EE-558 - Network tour of Data Science 
## Project: Stack Overflow survey network analysis

### Authors
Romain Choukroun, Matthias Leroy, Alain Milliet, Hector Parmantier

### Question

What's the best developer job like ? (depending on your own definition
of "best")

### Dataset

The dataset provided by [StackOverflow on
Kaggle](https://www.kaggle.com/stackoverflow/so-survey-2017/data) seems
to be a great start. It contains about fifty thousand answers from a
sample of the active StackOverflow population about a lot of questions,
namely 154. This means that we would have a tremendous insight into what
makes a programmer unique, but also can help us to answer a lot of
interesting questions.

### Project

#### Exploratory

Check the distributions of all useful features, outliers, quantiles.
Questions we could answer with the exploration:

-   Does salary equates to happiness/fulfilment in your job ?

-   For users not satisfied with their job, what should they change to
    be more satisfied (use closest correlated neighbor) ?

-   How much is Job Satisfaction linked to education ?

-   Are "gif" people more satisfied with their job compared to "jif"
    people ?
    
#### Metric
Derive metric to measure happiness/fulfillment

#### Pre-processing

Data cleaning, categorize values, check out their distribution,
selecting columns, removing bad values if needed.

#### Feature Extraction

PCA to check which are the features explaining the most variance.

#### Graph Analysis

The graph will be built the following way:

-   Users will be the nodes

-   Correlations (with a threshold) in-between users used as edges

#### Recommender System

The idea here would be to be able to recommend which of a set of users
best represents a set of given goals. To do so, we would simply check
which existing node is the closest to the artificial one that we create
for the chosen features a recruiter is looking for.


In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()
from subprocess import check_output
from sklearn.neighbors import kneighbors_graph
from sklearn import preprocessing
import networkx as nx
from helper_functions import plot_stud_prof, row_filter
pd.set_option('display.max_columns', None)

In [ ]:
stack = pd.read_csv("data/survey_results_public.csv")
kept_columns = ['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University', 'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'CompanySize', 'CompanyType', 'YearsProgram', 'YearsCodedJob', 'DeveloperType', 'WebDeveloperType', 'NonDeveloperType', 'CareerSatisfaction', 'JobSatisfaction', 'PronounceGIF', 'ProblemSolving', 'BuildingThings', 'LearningNewTech', 'BoringDetails', 'JobSecurity', 'DiversityImportant', 'FriendsDevelopers', 'WorkPayCare', 'ChallengeMyself', 'ImportantBenefits', 'ClickyKeys', 'Overpaid', 'TabsSpaces', 'EducationImportant', 'EducationTypes', 'SelfTaughtTypes', 'WorkStart', 'HaveWorkedLanguage', 'WantWorkLanguage', 'IDE', 'AuditoryEnvironment', 'Methodology', 'EquipmentSatisfiedMonitors', 'StackOverflowSatisfaction', 'StackOverflowFoundAnswer', 'StackOverflowCopiedCode', 'StackOverflowWhatDo', 'Gender', 'HighestEducationParents', 'Race', 'Salary', "ExpectedSalary"]
stack = stack[kept_columns]
stack.set_index("Respondent", inplace=True)
stack.head()

In [ ]:
stack = stack[stack.apply(lambda row: row_filter(stack,row), axis=1)]
prof_stack = stack[stack.Professional == "Professional developer"]
stud_stack = stack[stack.Professional == "Student"]

In [ ]:
metadata = pd.read_csv("data/survey_results_schema.csv")
metadata.head()

# Exploratory Analysis

In this section we will explore different columns of our dataframe to have an idea of what the population we have looks like. We decided to split the professionals and the students such that we can obtain 2 different recommender systems in function of what the recruiter is looking for. For certain features, it makes sense to compare the professionals and the students but for some others it makes more sense to do some statistics on the whole data.

## Professional
In this subsection we want to see the proportion of professionals and students we have in our data

In [ ]:
stack['Professional'].value_counts()[0:10].plot(kind='bar',figsize=(5,5))
plt.ylabel("Count")
plt.show()

On this graph, we can see the proportion of professionals and students in our dataset. As we can see, there are a lot more professionals than students.

## Country
We wanted to know from what country the people that answered to the survey come. As we have more professionals than students, we decided to take the 10 countries with the biggest number of professionals and from there we mapped the number of students for thes countries. 

In [ ]:
plot_stud_prof(prof_stack=prof_stack, stud_stack=stud_stack, column='Country', title="Country")

On this graph, we can see that the percentage of professionals is not always proportional to the percentage of students in the same country. The United States and India are the two most obvious ones. While 30% of professionals come from the USA, less than 20% of the students come from there. On the other hand, only 5% of professionals in our data come from India while nearly 15% of students come from there.

## Developer Type
This subsection concerns the type of developer we can find in our dataset. 

In [ ]:
DevTypes = pd.Series([devtype for sublist in [str(devtypes).replace(" ", "").split(";") for devtypes in stack['DeveloperType'].dropna()] for devtype in sublist])
DevTypes.value_counts(normalize=True)[0:10].plot(kind='bar',figsize=(7,7))
plt.ylabel("Ratio")
plt.show()

As an observation, we can say that the major part of the population in our dataset define themself as Web Developper. But this result does not give that much information as there are multiple types of Web developpers.

## Languages
As the previous section was about the type of developper we wanted to see if the programming languages correspond to the these types

In [ ]:
prof_languages = pd.Series([lang for sublist in [str(langs).replace(" ", "").split(";") for langs in prof_stack['HaveWorkedLanguage'].dropna()] for lang in sublist])
stud_languages = pd.Series([lang for sublist in [str(langs).replace(" ", "").split(";") for langs in stud_stack['HaveWorkedLanguage'].dropna()] for lang in sublist])
plot_stud_prof(prof=prof_languages, stud=stud_languages, title="Languages")

It looks like it corresponds pretty well as the most important programming language we find is directly linked to the Web Development.

## Job Satisfaction
In this case, we wanted to see how satisfied of their job the people in our dataset are. We compared the job satisfation with the career satisfaction.

In [ ]:
jobSat = stack['JobSatisfaction']/stack['JobSatisfaction'].max()
carrSat = stack['CareerSatisfaction']/stack['CareerSatisfaction'].max()
j = jobSat.value_counts(normalize=True).sort_index()
carr = carrSat.value_counts(normalize=True)
c = carr.loc[j.index]
df = pd.DataFrame([j, c])
df = df.T
df.columns = ["Job Satisfaction", "Career Satisfaction"]
df.plot.bar(figsize=(7,7))
plt.title('Satisfaction')
plt.ylabel("Count")
plt.show()

It is pretty clear on the graph above that the job satisfaction and the career satisfaction are directly linked. To remove all bias such as a person that votes 4 for the job and 8 for the career while another person votes 8 for the job and 4 for the career, we decided to look at the difference between these values and see if we have a lot of people having that much difference in their satisfactions.

In [ ]:
satisfaction = stack[['JobSatisfaction','CareerSatisfaction']]
satisfaction['Difference']= np.abs(satisfaction['JobSatisfaction']-satisfaction['CareerSatisfaction'])
satisfaction['Difference'].describe()

As we can see, in general people give approximately the same satisfaction score to their job and to their career. It means that the distribution seen above is correct.

## Diversity Important
In this section we will analyze if professionals and students give the same importance to the diversity at their job.

In [ ]:
plot_stud_prof(prof_stack=prof_stack, stud_stack=stud_stack, column='DiversityImportant', title="Diversity Importance")

We can pretty easily see that the two distributions are really close and it seems to not be different between professionals and students

# SEPARATION

## Salaries
Let's take a look at how the salaries compare for the whole dataset

In [ ]:
f, (box, dist) = plt.subplots(nrows=1, ncols=2, figsize=(10,6))
f.subplots_adjust(wspace=0.7)

sns.boxplot(prof_stack.Salary, orient='v', ax=box)
box.set_title("Box-plot of the total salaries")

usa_salary = prof_stack[prof_stack.Country == "United States"].Salary
usa_salary.rename("United States", inplace=True)
india_salary = prof_stack[prof_stack.Country == "India"].Salary
india_salary.rename("India", inplace=True)
india_salary.plot.kde(legend=True, ax=dist)
usa_salary.plot.kde(legend=True, ax=dist)
dist.set_title("Salary distribution")
dist.set_xlabel("Salary in USD")
plt.show()

As expected, the boxplot shows that we have a high breadth of salaries. If we compare the distribution of two different countries, we can see why, each country simply have their own revenue model for the profession. What about the students, do they their salary expectation meet the reality or not ?

In [ ]:
# We compare the student expectations to the young professionals
stud_stack.ExpectedSalary.plot(kind='kde', figsize=(10,8), color='r', legend=True)
prof_stack[(prof_stack.YearsProgram == "Less than a year")].Salary.plot(kind='kde', figsize=(7,7), legend=True)
plt.xlabel("Salary/Expected Salary in USD")
plt.title("Distribution of expected salary and salary for the students and professionals")
plt.show()

Indeed, they match quite nicely ! Students are not the highest dreamers after all.
## Personal Attributes
To get a final feel of the data, we take a look at the distribution of genders and races of the respondents.

In [ ]:
f, (race, gender) = plt.subplots(nrows=1, ncols=2, figsize=(10,6))
f.subplots_adjust(wspace=0.7)

prof_stack.Race.value_counts(normalize=True)[0:6].plot(kind='bar', ax=race)
race.set_title("Distribution of the races")
race.set_ylabel("Ratio")

prof_stack.Gender.value_counts(normalize=True)[0:4].plot(kind='bar', ax=gender)
gender.set_title("Distribution of the genders")
gender.set_ylabel("Ratio")

plt.show()

With no surprise, the majority of respondents are white males.
## Education types
Another interesting thing to know is how these developers learned to code. Do the students and professionals learned the same way.

In [ ]:
prof_education = pd.Series([ed for sublist in [str(educs).replace(" ", "").split(";") for educs in prof_stack['EducationTypes'].dropna()] for ed in sublist])
stud_education = pd.Series([ed for sublist in [str(educs).replace(" ", "").split(";") for educs in stud_stack['EducationTypes'].dropna()] for ed in sublist])
plot_stud_prof(prof=prof_education, stud=stud_education, title="Education types")

Again without any surprise, the distributions are quite the same, appart from the ones linked to the job/student life, for example students are more likely to learn using online courses and professional using on-the-job training.

## GIF vs JIF

In [ ]:
import matplotlib.pyplot as plt
temp = stack[["PronounceGIF", "Salary"]].dropna(how='any')
temp = temp.set_index("PronounceGIF")
gif = temp.loc['With a hard "g," like "gift"'].Salary.values
jif = temp.loc['With a soft "g," like "jiff"'].Salary.values
tr = pd.DataFrame()
tr['gif'] = pd.Series(gif)
filling = np.empty((6081))
filling[:] = np.nan
to_add = np.append(jif, filling)
tr['jif'] = pd.Series(to_add)
plot = sns.boxplot(data=tr, orient="v",)
plt.ylabel("Salary")
plt.title("Distribution of salary for the gif and jif populations")
plt.show()

print(tr.describe())

# ECRIS LA MATTHIAS

In [ ]:
prof_stack.isnull().sum()

In [ ]:
stud_stack.isnull().sum()

In [ ]:
important_features_prof = ['Professional', 'ProgramHobby', 'Country', 'University', 'FormalEducation', 'MajorUndergrad', 'CompanyType'
                           ,'YearsCodedJob', 'YearsProgram', 'DeveloperType', 'CareerSatisfaction', 'JobSatisfaction', 'Overpaid',
                           'WorkStart', 'HaveWorkedLanguage', 'WantWorkLanguage', 'AuditoryEnvironment', 'Salary']

important_features_stud = ['Professional', 'ProgramHobby', 'Country', 'University', 'FormalEducation', 'YearsProgram','WorkStart',
                           'ClickyKeys', 'HaveWorkedLanguage', 'WantWorkLanguage', 'AuditoryEnvironment', 'ExpectedSalary']

final_prof_stack = prof_stack[important_features_prof].copy()
final_stud_stack = stud_stack[important_features_stud].copy()

In [ ]:
final_prof_stack = final_prof_stack.dropna()
final_prof_stack.shape

In [ ]:
final_stud_stack = final_stud_stack.dropna()
final_stud_stack.shape

In [ ]:
label_prof_stack = final_prof_stack.copy()
label_stud_stack = final_stud_stack.copy()

for c in important_features_prof[:-1]:
    le = preprocessing.LabelEncoder()
    le.fit(label_prof_stack[c])
    label_prof_stack[c] = le.transform(label_prof_stack[c])

for c in important_features_stud[:-1]:
    le = preprocessing.LabelEncoder()
    le.fit(label_stud_stack[c])
    label_stud_stack[c] = le.transform(label_stud_stack[c])

In [ ]:
prof_neighbors = kneighbors_graph(label_prof_stack, 98, mode='distance', include_self=True)
prof_rbf = prof_neighbors.copy()
prof_rbf.data = np.exp(- prof_rbf.data ** 2 / (2. * np.mean(prof_rbf.data) ** 2))
del prof_neighbors

stud_neighbors = kneighbors_graph(label_stud_stack, 43, mode='distance', include_self=True)
stud_rbf = stud_neighbors.copy()
stud_rbf.data = np.exp(- stud_rbf.data ** 2 / (2. * np.mean(stud_rbf.data) ** 2))
del stud_neighbors


In [ ]:
G_prof = nx.from_scipy_sparse_matrix(prof_rbf,edge_attribute='similarity')
pos_prof = nx.spring_layout(G_prof)
nx.draw_networkx_nodes(G_prof, pos_prof, node_size=7)
plt.show()

In [ ]:
G_stud = nx.from_scipy_sparse_matrix(stud_rbf,edge_attribute='similarity')
pos_stud = nx.spring_layout(G_stud)
nx.draw_networkx_nodes(G_stud, pos_stud, node_size=7)
plt.show()